In [1]:
# Notebook for taking the csv and converting it to the correct format for BQ data schema
# Remember date_added is now a timestamp!

In [2]:
import pandas as pd
import os

In [18]:
os.path.dirname(os.path.abspath("__file__"))

'/home/ed/code/Eatkin/bookmarks_interface/transition_to_bq'

In [21]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))), "bookmarks-414106-82ffd7e77bda.json")
os.environ.get("GOOGLE_APPLICATION_CREDENTIALS")

'/home/ed/code/Eatkin/bookmarks_interface/bookmarks-414106-82ffd7e77bda.json'

In [3]:
csv_filename = os.path.join(os.path.dirname("__file__"), 'bookmarks.csv')
df = pd.read_csv(csv_filename)

In [4]:
df.head()

,Bookmark_ID,Title,Category,URL,Description,Date_Added,Tags
0,1,Cosmetics bag,Sewing Patterns,https://www.youtube.com/watch?v=TW9t0HTK_7E&fe...,FREE PATTERN HERE - http://so-sew-easy.com/eas...,2016-04-10 14:32:02,"sewing, crafts"
1,2,Underwear,Sewing Patterns,http://verypurpleperson.com/2013/01/tutorial-s...,Tutorial: Sewing Panties &#8211; verypurpleperson,2016-04-10 14:32:27,"sewing, clothes, crafts"
2,11,Crumpet recipe | RecipeTin Eats,Recipes,https://www.recipetineats.com/crumpet-recipe/,This is the crumpet recipe released by Warburt...,2023-04-10 10:44:13,"breakfast, recipe, food, bread"
3,101,Feedly,RSS,https://feedly.com/i/my,Welcome to Feedly — the platform where busines...,2023-07-09 20:45:27,"rss, blog"
4,102,Starter Packs — You Need Feeds,RSS,https://www.youneedfeeds.com/starter-packs,"Interested in web feeds and RSS, but not sure ...",2023-07-10 13:29:54,"rss, reference"


In [5]:
# First we'll make our category IDs table
unique_categories = df['Category'].unique()

In [6]:
# Create a dataframe from this new array with an autoincrementing index
df_categories = pd.DataFrame(unique_categories, columns=['Category'])

In [7]:
df_categories['Category_ID'] = df_categories.index

In [8]:
df_categories.head()


,Category,Category_ID
0,Sewing Patterns,0
1,Recipes,1
2,RSS,2
3,Blogs,3
4,Socials,4


In [27]:
# Rename Categories to Name
df_categories = df_categories.rename(columns={'Category': 'Name'})

In [28]:
# Cool let's send it to BQ
from google.cloud import bigquery
project_id = "bookmarks-414106"
dataset_id = "bookmarks_data"
table_name = 'categories'

In [29]:
# Let's send that shit
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

table_id = f"{project_id}.{dataset_id}.{table_name}"

client.load_table_from_dataframe(df_categories, table_id)

LoadJob<project=bookmarks-414106, location=europe-west2, id=bfcc7127-cc16-452b-8ae0-d4b7d3cad877>

In [105]:
df_tags = df['Tags']

In [106]:
df_tags = pd.DataFrame(df_tags)

In [107]:
df_tags.head()

,Tags
0,"sewing, crafts"
1,"sewing, clothes, crafts"
2,"breakfast, recipe, food, bread"
3,"rss, blog"
4,"rss, reference"


In [108]:
df_tags['Tags'] = df_tags['Tags'].str.split(", ")

In [110]:
df_tags = df_tags.explode('Tags')

In [112]:
df_tags = df_tags.drop_duplicates()

In [115]:
df_tags.shape

(158, 1)

In [116]:
len(df_tags['Tags'].unique())

158

In [117]:
# Now create an index
df_tags['Tag_ID'] = [i for i in range(158)]

In [133]:
df_tags.head()

,Name,Tag_ID
0,sewing,0
0,crafts,1
1,clothes,2
2,breakfast,3
2,recipe,4


In [132]:
df_tags = df_tags.rename({"Tags": "Name"}, axis=1)

In [134]:
table_name = "tags"

# Send to BQ as before
table_id = f"{project_id}.{dataset_id}.{table_name}"
client.load_table_from_dataframe( df_tags, table_id)

LoadJob<project=bookmarks-414106, location=europe-west2, id=833cd4c7-014f-4d01-93bc-919ed8302bfe>

In [63]:
bookmark_table_columns = ["Bookmark_ID", "Title", "URL", "Description", "Date_Added", "Category"]
bookmarks_df = df[bookmark_table_columns]
bookmarks_df.head()

,Bookmark_ID,Title,URL,Description,Date_Added,Category
0,1,Cosmetics bag,https://www.youtube.com/watch?v=TW9t0HTK_7E&fe...,FREE PATTERN HERE - http://so-sew-easy.com/eas...,2016-04-10 14:32:02,Sewing Patterns
1,2,Underwear,http://verypurpleperson.com/2013/01/tutorial-s...,Tutorial: Sewing Panties &#8211; verypurpleperson,2016-04-10 14:32:27,Sewing Patterns
2,11,Crumpet recipe | RecipeTin Eats,https://www.recipetineats.com/crumpet-recipe/,This is the crumpet recipe released by Warburt...,2023-04-10 10:44:13,Recipes
3,101,Feedly,https://feedly.com/i/my,Welcome to Feedly — the platform where busines...,2023-07-09 20:45:27,RSS
4,102,Starter Packs — You Need Feeds,https://www.youneedfeeds.com/starter-packs,"Interested in web feeds and RSS, but not sure ...",2023-07-10 13:29:54,RSS


In [64]:
bookmarks_df["Date_Added"] = pd.to_datetime(bookmarks_df["Date_Added"])

/tmp/ipykernel_17213/1230923170.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bookmarks_df["Date_Added"] = pd.to_datetime(bookmarks_df["Date_Added"])


In [68]:
# Merge on categories_df
bookmarks_df = pd.merge(left=bookmarks_df, right=df_categories, left_on="Category", right_on="Name")

In [69]:
# Drop both Category and Name
bookmarks_df.drop(labels=['Name', 'Category'], inplace=True, axis=1)

In [70]:
bookmarks_df.head()

,Bookmark_ID,Title,URL,Description,Date_Added,Category_ID
0,1,Cosmetics bag,https://www.youtube.com/watch?v=TW9t0HTK_7E&fe...,FREE PATTERN HERE - http://so-sew-easy.com/eas...,2016-04-10 14:32:02,0
1,2,Underwear,http://verypurpleperson.com/2013/01/tutorial-s...,Tutorial: Sewing Panties &#8211; verypurpleperson,2016-04-10 14:32:27,0
2,3,Bra,http://www.instructables.com/id/Bra/,"Bra: A bra is a bit more advanced project, but...",2016-04-10 18:29:46,0
3,4,Boxer Shorts,http://www.measuretwicecutonce.com.au/product/...,PATTERN DESCRIPTION Like their namesake Mr Dar...,2016-04-11 19:52:37,0
4,5,Modern Sewing Patterns,http://m-sewing.com/,Sewing patterns catalog of stylish clothing. N...,2016-04-16 21:12:44,0


In [71]:
table_name = "bookmarks"

# Send to BQ as before
table_id = f"{project_id}.{dataset_id}.{table_name}"
client.load_table_from_dataframe(bookmarks_df, table_id)

LoadJob<project=bookmarks-414106, location=europe-west2, id=465cfe68-d6b1-41a6-9db2-a50518b16474>

In [72]:
# Create the bookmarktags table
df.head()

,Bookmark_ID,Title,Category,URL,Description,Date_Added,Tags
0,1,Cosmetics bag,Sewing Patterns,https://www.youtube.com/watch?v=TW9t0HTK_7E&fe...,FREE PATTERN HERE - http://so-sew-easy.com/eas...,2016-04-10 14:32:02,"sewing, crafts"
1,2,Underwear,Sewing Patterns,http://verypurpleperson.com/2013/01/tutorial-s...,Tutorial: Sewing Panties &#8211; verypurpleperson,2016-04-10 14:32:27,"sewing, clothes, crafts"
2,11,Crumpet recipe | RecipeTin Eats,Recipes,https://www.recipetineats.com/crumpet-recipe/,This is the crumpet recipe released by Warburt...,2023-04-10 10:44:13,"breakfast, recipe, food, bread"
3,101,Feedly,RSS,https://feedly.com/i/my,Welcome to Feedly — the platform where busines...,2023-07-09 20:45:27,"rss, blog"
4,102,Starter Packs — You Need Feeds,RSS,https://www.youneedfeeds.com/starter-packs,"Interested in web feeds and RSS, but not sure ...",2023-07-10 13:29:54,"rss, reference"


In [97]:
# First we have to explode the tags column
bookmark_tags_df = df.copy()
bookmark_tags_df['Tags'] = bookmark_tags_df['Tags'].str.split(', ')

In [98]:
bookmark_tags_df = bookmark_tags_df.explode('Tags')

In [99]:
# Drop all but Tags and Bookmark_ID
bookmark_tags_df = bookmark_tags_df[['Bookmark_ID', 'Tags']]

In [135]:
bookmark_tags_df.head()

,Bookmark_ID,Tags
0,1,sewing
0,1,crafts
1,2,sewing
1,2,clothes
1,2,crafts


In [136]:
# Merge with tags_df
bookmark_tags_df = pd.merge(left=bookmark_tags_df, right=df_tags, left_on="Tags", right_on="Name")

In [137]:
bookmark_tags_df

,Bookmark_ID,Tags,Name,Tag_ID
0,1,sewing,sewing,0
1,2,sewing,sewing,0
2,3,sewing,sewing,0
3,4,sewing,sewing,0
4,5,sewing,sewing,0
...,...,...,...,...
1064,88,slack,slack,154
1065,97,gambling,gambling,155
1066,97,horse-racing,horse-racing,156
1067,98,pixel-art,pixel-art,157


In [138]:
# Now drop the Tags and Name columns
bookmark_tags_df.drop(labels=['Tags', 'Name'], inplace=True, axis=1)

In [139]:
bookmark_tags_df.head()

,Bookmark_ID,Tag_ID
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [140]:
# Npw send to BQ
table_name = "bookmarktags"

table_id = f"{project_id}.{dataset_id}.{table_name}"
client.load_table_from_dataframe(bookmark_tags_df, table_id)

LoadJob<project=bookmarks-414106, location=europe-west2, id=bfd91183-b7dd-418b-b653-ad4bce101f27>